In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [2]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft, colors
from skimpy import functional as F

In [3]:
level = minecraft.SkimpyMinecraftLevel.load("../data/pistonhouse.pickle.gz")

In [19]:
level.to_map_image()

AttributeError: module 'nbt.chunk' has no attribute 'block_ids'

In [4]:
t = level.megatensor()

In [5]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>21319363
1=>14689733
2=>115759
3=>1534780
4=>3200
5=>6254
6=>2
7=>1000428
8=>300
9=>2064869
10=>54569
11=>81511
12=>434809
13=>454927
14=>10341
15=>97217
16=>171470
17=>12522
18=>134545
19=>0


In [6]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
for i in range(1, 256):
    #r, g, b = colors.color_for_id(i)
    #config[i] = skimpy_3d.ColorVoxel(r, g, b)
    config[i] = skimpy_3d.ColorVoxel(
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255),
    )

In [7]:
%%time
mesh = skimpy_3d.generate_mesh(config, t._tensor)

Wall time: 1.33 s


In [8]:
len(mesh.triangles)

14135076

In [9]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [10]:
geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='float32').reshape(-1, 3) / 255.0,
        ),
    },
)

In [11]:
geometry.exec_three_obj_method("computeVertexNormals")

In [12]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [13]:
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(9423384, 3), dtype=…

In [17]:
camera = PerspectiveCamera(position=[300, 50, 1200], fov=20)

point_light = PointLight(color='#ffffff', position=[300, 100, 1200])
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light])

In [18]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)

Renderer(background='white', camera=PerspectiveCamera(fov=20.0, position=(300.0, 50.0, 1200.0), projectionMatr…